In [2]:
import numpy as np
import gym
import ppaquette_gym_doom
from ppaquette_gym_doom.wrappers import SetResolution, ToDiscrete
from gym.wrappers import SkipWrapper
from gym import wrappers

# (see https://github.com/ppaquette/gym-doom/blob/master/ppaquette_gym_doom/doom_basic.py)
def create_env(seed=None, monitory_directory=None):
    env_spec = gym.spec('ppaquette/DoomBasic-v0')
    env_spec.id = 'DoomBasic-v0'
    env = env_spec.make()

    if monitory_directory is not None:
        env = wrappers.Monitor(
            env, directory=monitory_directory, force=True, mode='training',
            video_callable=lambda episode_id: episode_id % 100 == 0)

    if seed is not None:
        env.seed(seed)

    return SetResolution('200x150')(
        SkipWrapper(repeat_count=4)(
        ToDiscrete('minimal')(env)))

env = create_env()
WIDTH, HEIGHT = env.screen_width, env.screen_height

NOOP, SHOOT, RIGHT, LEFT = 0, 1, 2, 3

## Collecting experiences

In [3]:
from collections import namedtuple
import operator

SARE = namedtuple('SAR', ['state', 'action', 'reward', 'end'])


def generate_sares(env, agent, episode_count=100):
    reward = 0
    done = False

    for i in range(episode_count):
        observation = env.reset()
        while True:
            action = agent.act(observation, reward, done)
            new_observation, reward, done, _ = env.step(action)
            yield SARE(observation, action, reward, done)
            
            if done:
                break
            else:
                observation = new_observation

def episode_sares(env, agent, episode_count=100):
    sares = list(generate_sares(env, agent, episode_count))
    print('average reward per episode = {}'.format(
        sum(r for _, _, r, _ in sares) / float(sum(e for _, _, _, e in sares))))
    return sares

        
def to_experiences(sares, only_n_misses=100):
    experiences = [
        (previous_s, a, r, next_s, end)
        for (previous_s, a, r, end), (next_s, _, _, _) in zip(sares[:-1], sares[1:])
    ]

    # simplistic experience prioritization
    shuffled_exps = experiences if only_n_misses is None\
        else random.choices(experiences, k=only_n_misses) + [e for e in experiences if e[2] > 0]
    random.shuffle(shuffled_exps)

    prev_frames, actions, rewards, next_frames, is_ends = zip(*shuffled_exps)
    prev_frames = np.asarray(prev_frames)
    next_frames = np.asarray(next_frames)
    actions = np.asarray(actions)
    rewards = np.asarray(rewards)
    is_ends = np.asarray(is_ends)
    
    print('Training on {}/{} positive/total out of {} 1-step experiences with actions distribution {}'.format(
        np.sum(rewards>=0),
        len(rewards),
        len(experiences),
        np.bincount(actions)))
    
    return (prev_frames, next_frames, actions, rewards, is_ends)

## Deep Q-learning

In [4]:
import tensorflow as tf
from keras import backend as K

from keras.layers import Dense, Convolution2D, Flatten, Activation
from keras.models import Sequential
from keras.optimizers import Adam

sess = tf.InteractiveSession()
K.set_session(sess)

def create_q_model(conv1_weights=None, conv2_weights=None, dense1_weights=None, dense2_weights=None):
    model = Sequential()

    model.add(
        Convolution2D(nb_filter=2, nb_row=6, nb_col=6, subsample=(1, 1), border_mode='valid', weights=conv1_weights,
            input_shape=[HEIGHT, WIDTH, 3], dim_ordering='tf'))
    model.add(Activation('relu'))
    model.add(
        Convolution2D(nb_filter=4, nb_row=2, nb_col=2, subsample=(1, 1), weights=conv2_weights))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(64, init='normal', weights=dense1_weights))
    model.add(Activation('relu'))
    model.add(Dense(4, init='normal', weights=dense2_weights))
    model.compile(loss='mse', optimizer=Adam())
    
    return model

acting_model = create_q_model()
target_model = create_q_model()

def copy_model(model):
    conv1_weights =  [w.eval() for w in model.layers[0].weights]
    conv2_weights = [w.eval() for w in model.layers[2].weights]
    dense1_weights = [w.eval() for w in model.layers[5].weights]
    dense2_weights = [w.eval() for w in model.layers[7].weights]
    return create_q_model(conv1_weights, conv2_weights, dense1_weights, dense2_weights)

acting_model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 145, 195, 2)   218         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 145, 195, 2)   0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 4, 194, 1)     2324        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 4, 194, 1)     0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [5]:
def sares_to_input_targets(model, sares, gamma = .99, reward_clip=5, only_n_misses=100):
    
    prev_frames, next_frames, actions, rewards, is_ends = to_experiences(sares, only_n_misses)
    
    n_samples = len(actions)
    clipped_rewards = np.clip(rewards, -np.inf, reward_clip)
    
    # Transcription of the Q-learning target formula
    targets = clipped_rewards + gamma * (1 - is_ends) * model.predict(next_frames).max(axis=1)

    target_action_rewards = model.predict(prev_frames)
    target_action_rewards[np.arange(n_samples), actions] = targets

    return prev_frames, target_action_rewards

## Training

In [6]:
import random

class EpsilonGreedyQAgent(object):
    def __init__(self, model, epsilon=.1):
        self.model = model
        self.epsilon = epsilon

    def act(self, observation, reward, done):
        if random.uniform(0, 1) <= self.epsilon:
            return random.choice([NOOP, SHOOT, LEFT, RIGHT])
        else:
            return self.model.predict(observation[np.newaxis])[0].argmax()


N_BATCHES = 200
N_BATCHED_EPISODES = 10
UPDATE_TARGET_EVERY_N_BACTHES = 2
MINI_BATCH_SIZE = 32
REWARD_CLIP = 5
ONLY_N_MISSES = 200

directory = 'tmp/DoomBasic_old_qlearning'
env = create_env(monitory_directory=directory)

for _ in range(N_BATCHES):
    for _ in range(UPDATE_TARGET_EVERY_N_BACTHES):
        sares = episode_sares(env, EpsilonGreedyQAgent(acting_model, epsilon=.1), N_BATCHED_EPISODES)
        if (sum(r for _, _, r, _ in sares) / N_BATCHED_EPISODES) > 80:
            break
        prev_frames, target_action_rewards = sares_to_input_targets(target_model, sares, reward_clip=REWARD_CLIP, only_n_misses=ONLY_N_MISSES)
        acting_model.fit(x=prev_frames, y=target_action_rewards, batch_size=MINI_BATCH_SIZE, nb_epoch=1, verbose=0)
    
    target_model = copy_model(acting_model)

# final greedy episodes
sares = episode_sares(env, EpsilonGreedyQAgent(acting_model, epsilon=0), episode_count=100)

env.close()
gym.upload(directory, api_key='sk_bNZUvCfkTfabQCoKoKbjFA')

[2017-03-12 11:51:38,976] DEPRECATION WARNING: env.spec.timestep_limit has been deprecated. Replace your call to `env.spec.timestep_limit` with `env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')`. This change was made 12/28/2016 and is included in version 0.7.0
[2017-03-12 11:51:38,977] Clearing 2 monitor files from previous run (because force=True was provided)
[2017-03-12 11:51:39,912] Starting new video recorder writing to /Users/gui/Dev/rl-study/tmp/DoomBasic_old_qlearning/openaigym.video.0.36214.video000000.mp4


average reward per episode = -242.7
Training on 3/203 positive/total out of 518 1-step experiences with actions distribution [172  19   5   7]
average reward per episode = -343.4
Training on 2/201 positive/total out of 696 1-step experiences with actions distribution [189   5   5   2]
average reward per episode = -313.4
Training on 2/201 positive/total out of 631 1-step experiences with actions distribution [ 47   7 133  14]
average reward per episode = -359.5
Training on 0/200 positive/total out of 699 1-step experiences with actions distribution [  5   5  38 152]
average reward per episode = -292.6
Training on 3/203 positive/total out of 630 1-step experiences with actions distribution [  6  10  73 114]
average reward per episode = -358.0
Training on 0/200 positive/total out of 699 1-step experiences with actions distribution [  5   6 173  16]
average reward per episode = -360.5
Training on 0/200 positive/total out of 699 1-step experiences with actions distribution [  6  10  35 149]

[2017-03-12 11:53:23,552] Starting new video recorder writing to /Users/gui/Dev/rl-study/tmp/DoomBasic_old_qlearning/openaigym.video.0.36214.video000100.mp4


average reward per episode = -229.6
Training on 4/204 positive/total out of 527 1-step experiences with actions distribution [  5   7  78 114]
average reward per episode = -300.8
Training on 2/202 positive/total out of 631 1-step experiences with actions distribution [  7   8  86 101]
average reward per episode = -360.5
Training on 0/200 positive/total out of 699 1-step experiences with actions distribution [102   5  38  55]
average reward per episode = -358.5
Training on 0/200 positive/total out of 699 1-step experiences with actions distribution [67  3 55 75]
average reward per episode = -332.4
Training on 1/201 positive/total out of 666 1-step experiences with actions distribution [158   3   7  33]
average reward per episode = -358.5
Training on 0/200 positive/total out of 699 1-step experiences with actions distribution [  3   5  86 106]
average reward per episode = -222.1
Training on 5/204 positive/total out of 506 1-step experiences with actions distribution [82 11 43 68]
average

[2017-03-12 11:55:05,654] Starting new video recorder writing to /Users/gui/Dev/rl-study/tmp/DoomBasic_old_qlearning/openaigym.video.0.36214.video000200.mp4


...

[2017-03-12 13:05:46,818] Starting new video recorder writing to /Users/gui/Dev/rl-study/tmp/DoomBasic_old_qlearning/openaigym.video.0.36214.video003900.mp4


average reward per episode = -119.5
Training on 13/206 positive/total out of 305 1-step experiences with actions distribution [ 7 70 43 86]
average reward per episode = -58.1
Training on 13/206 positive/total out of 241 1-step experiences with actions distribution [38 36 43 89]
average reward per episode = -106.8
Training on 11/206 positive/total out of 293 1-step experiences with actions distribution [ 7 85 53 61]
average reward per episode = -66.1
Training on 10/206 positive/total out of 241 1-step experiences with actions distribution [ 4 66 46 90]
average reward per episode = 13.4
Training on 19/208 positive/total out of 131 1-step experiences with actions distribution [33 93 31 51]
average reward per episode = -63.1
Training on 11/207 positive/total out of 238 1-step experiences with actions distribution [95 38 14 60]
average reward per episode = -66.4
Training on 12/207 positive/total out of 258 1-step experiences with actions distribution [  6  44  34 123]
average reward per epi

[2017-03-12 13:07:41,927] Starting new video recorder writing to /Users/gui/Dev/rl-study/tmp/DoomBasic_old_qlearning/openaigym.video.0.36214.video004000.mp4
[2017-03-12 13:08:02,906] Finished writing results. You can upload them to the scoreboard via gym.upload('/Users/gui/Dev/rl-study/tmp/DoomBasic_old_qlearning')
[2017-03-12 13:08:02,913] [DoomBasic-v0] Uploading 4100 episodes of training data


average reward per episode = -17.81


[2017-03-12 13:08:13,533] [DoomBasic-v0] Uploading videos of 41 training episodes (10178788 bytes)
[2017-03-12 13:09:45,458] [DoomBasic-v0] Creating evaluation object from tmp/DoomBasic_old_qlearning with learning curve and training video
[2017-03-12 13:09:45,825] 
****************************************************
You successfully uploaded your evaluation on DoomBasic-v0 to
OpenAI Gym! You can find it at:

    https://gym.openai.com/evaluations/eval_hfhBF7xdSf2tRV1Jsv0MXg

****************************************************
